# Hikingbook 桃山路線 GPX 自動下載器
使用 Selenium 自動化登入與 GPX 下載，適用於桃山相關路線活動

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium_stealth import stealth
import time, os
from pathlib import Path
from dotenv import load_dotenv

In [2]:
# === 初始化下載資料夾與帳密 ===
download_dir = Path.cwd() / "下載GPX"
download_dir.mkdir(exist_ok=True)
load_dotenv()
EMAIL = os.getenv("HIKINGBOOK_EMAIL")
PASSWORD = os.getenv("HIKINGBOOK_PASSWORD")

In [3]:
# === 初始化瀏覽器 ===
options = Options()
prefs = { "download.default_directory": str(download_dir) }
options.add_experimental_option("prefs", prefs)
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 5)
stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True)

In [4]:
def login_if_required():
    try:
        login_link = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.btn.btn-secondary[href^='/login']")))
        driver.execute_script("arguments[0].click();", login_link)
        time.sleep(5)
        email_login_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn.btn-secondary[data-bs-target='#account-form']")))
        driver.execute_script("arguments[0].click();", email_login_btn)
        time.sleep(5)
        email_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#account-form input[type='email']")))
        password_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#account-form input[type='password']")))
        email_input.send_keys(EMAIL)
        password_input.send_keys(PASSWORD)
        submit_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#account-form button.btn.btn-primary")))
        driver.execute_script("arguments[0].click();", submit_btn)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='/account']")))
    except Exception as e:
        print(f"登入失敗：{e}")

In [5]:
#關閉彈出視窗
def close_modal_if_exists():
    try:
        close_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".modal .btn-close")))
        close_btn.click()
    except TimeoutException:
        pass

In [6]:
def handle_download_modal():
    try:
        continue_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "a.btn.btn-primary[href$='/download']"))
        )
        driver.execute_script("arguments[0].click();", continue_btn)
        time.sleep(3)
    except TimeoutException:
        print("⚠️ 沒有『繼續下載』按鈕")

In [7]:
def wait_for_gpx_download(timeout=5):
    start_time = time.time()
    while time.time() - start_time < timeout:
        files = os.listdir(download_dir)
        if any(f.endswith('.gpx') for f in files) and not any(f.endswith('.crdownload') for f in files):
            return
        time.sleep(1)
    print("❌ GPX 下載逾時")

In [8]:
# === 執行 GPX 自動下載流程 (支援多頁) ===
BASE_URL = "https://zh-tw.hikingbook.net/explore/activities?regions=Taiwan&durations=0-3&durations=3-6&durations=6-24&query=%E6%A1%83%E5%B1%B1&order=latest"
driver.get(BASE_URL)

# 總共要處理 70 頁
for page in range(1, 71):
    print(f"📄 開始處理第 {page} 頁...")
    
    try:
        # 等待目前頁面的卡片都載入完成
        # 這裡加入一個短暫的 sleep，確保 AJAX 內容都已刷新
        time.sleep(2) 
        cards = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card.text-bg-white.rounded-2.position-relative.border-1")))
        print(f"   - 第 {page} 頁找到 {len(cards)} 個項目。")

        # 迴圈處理當前頁面的每一個項目
        for idx, card in enumerate(cards):
            try:
                # 取得卡片的連結並在新分頁中開啟
                link = card.find_element(By.CSS_SELECTOR, "a.stretched-link").get_attribute("href")
                driver.execute_script("window.open(arguments[0]);", link)
                driver.switch_to.window(driver.window_handles[1])

                # --- 在新分頁中執行下載 ---
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.card-body")))
                login_if_required()
                close_modal_if_exists()
                download_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn.btn-primary.flex-fill")))
                driver.execute_script("arguments[0].click();", download_btn)
                handle_download_modal()
                wait_for_gpx_download()
                # --- 下載完成 ---

                print(f"   ✅ 第 {page} 頁，第 {idx+1} 個項目下載成功。")

            except Exception as e:
                print(f"   ⚠️ 第 {page} 頁，第 {idx+1} 筆項目處理錯誤：{e}")
            
            finally:
                # 無論成功或失敗，都關閉分頁並切換回主視窗
                if len(driver.window_handles) > 1:
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

        # --- 當前頁面所有項目處理完畢 ---

        # 如果還不是最後一頁，就點擊「下一頁」按鈕
        if page < 70:
            print(f"   -> 準備翻至第 {page + 1} 頁...")
            # 使用您提供的 XPath 定位下一頁按鈕
            next_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//i[@class='bi bi-chevron-right']")))
            # 捲動到按鈕可見的位置再點擊，增加穩定性
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_btn)
            time.sleep(0.5) # 捲動後短暫等待
            driver.execute_script("arguments[0].click();", next_btn)
            # 給予足夠時間讓下一頁的內容載入
            time.sleep(3) 
        
    except Exception as e:
        print(f"❌ 第 {page} 頁處理時發生嚴重錯誤，可能無法繼續: {e}")
        break

print("\n🎉 所有頁面處理完畢。")


📄 開始處理第 1 頁...
   - 第 1 頁找到 11 個項目。
登入失敗：Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff7b5f66f75+76917]
	GetHandleVerifier [0x0x7ff7b5f66fd0+77008]
	(No symbol) [0x0x7ff7b5d19dea]
	(No symbol) [0x0x7ff7b5d70256]
	(No symbol) [0x0x7ff7b5d7050c]
	(No symbol) [0x0x7ff7b5dc3887]
	(No symbol) [0x0x7ff7b5d984af]
	(No symbol) [0x0x7ff7b5dc065c]
	(No symbol) [0x0x7ff7b5d98243]
	(No symbol) [0x0x7ff7b5d61431]
	(No symbol) [0x0x7ff7b5d621c3]
	GetHandleVerifier [0x0x7ff7b623d2ad+3051437]
	GetHandleVerifier [0x0x7ff7b6237903+3028483]
	GetHandleVerifier [0x0x7ff7b625589d+3151261]
	GetHandleVerifier [0x0x7ff7b5f8183e+185662]
	GetHandleVerifier [0x0x7ff7b5f896ff+218111]
	GetHandleVerifier [0x0x7ff7b5f6faf4+112628]
	GetHandleVerifier [0x0x7ff7b5f6fca9+113065]
	GetHandleVerifier [0x0x7ff7b5f56c78+10616]
	BaseThreadInitThunk [0x0x7ffb0c8fe8d7+23]
	RtlUserThreadStart [0x0x7ffb0e1dc34c+44]

   ✅ 第 1 頁，第 1 個項目下載成功。
登入失敗：Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff7b5f66f75+76917]
	GetHandleVerifier